In [1]:

from src.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from src.Recommenders.Neural.MultVAE_PyTorch_Recommender import MultVAERecommender_PyTorch
from src.Recommenders.MatrixFactorization.PureSVDRecommender import ScaledPureSVDRecommender
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDItemRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

import numpy as np
import random
import os

os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))


SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)


import pandas as pd
import scipy.sparse as sps

dataframe_items=pd.read_csv("Data/data_ICM_metadata.csv",low_memory=True)

icm=sps.csr_matrix((dataframe_items["data"].values,(dataframe_items["item_id"].values.astype(int),dataframe_items["feature_id"].values)))

urm_train = sps.load_npz('Data/URM_train.npz')
urm_test = sps.load_npz('Data/URM_test.npz')
urm_train_stacked=sps.vstack([urm_train,icm.T])

ModuleNotFoundError: No module named 'final_hybrid_submission'

In [3]:
import sqlite3
database_path='optuna_hybrid_linear_all_models.db'
conn=sqlite3.connect(database_path)
cursor=conn.cursor()
cursor.execute(""" UPDATE trials SET state='WAITING' WHERE state='RUNNING' or state='FAIL' """)
conn.commit()
cursor.close()

In [4]:
alpha_p3=0.91826869357383
urm_train_p3_stacked=sps.vstack([urm_train*alpha_p3,icm.T*(1-alpha_p3)])

In [5]:
slim_recommender=SLIMElasticNetRecommender(urm_train)
pure_svd_recommender=PureSVDItemRecommender(urm_train)
rp3beta_recommender=RP3betaRecommender(urm_train)
rp3beta_stacked_recommender=RP3betaRecommender(urm_train_stacked)
scaled_svd_recommender= ScaledPureSVDRecommender(urm_train)
multVAE_recommender=MultVAERecommender_PyTorch(urm_train)
itemKNN_recommender=ItemKNNCFRecommender(urm_train)
p3alpha_recommender=P3alphaRecommender(urm_train)
p3alpha_stacked_recommender=P3alphaRecommender(urm_train_p3_stacked)
ials_recommender=IALSRecommender(urm_train)
ials_stacked_recommender=IALSRecommender(urm_train_stacked)
nmf_recommender=NMFRecommender(urm_train)

In [6]:
folder_path='trained_models/train/'
slim_recommender.load_model(folder_path=folder_path,file_name='SLIMElasticNetRecommender.zip')
pure_svd_recommender.load_model(folder_path=folder_path,file_name='best_pure_svd.zip')
multVAE_recommender.load_model(folder_path=folder_path,file_name='best_MultVAE.zip')
scaled_svd_recommender.load_model(folder_path=folder_path,file_name='best_scaled_svd.zip')
rp3beta_recommender.load_model(folder_path=folder_path,file_name='best_rp3beta.zip')
rp3beta_stacked_recommender.load_model(folder_path=folder_path,file_name='best_stacked_rp3beta.zip')

itemKNN_recommender.load_model(folder_path=folder_path,file_name='best_itemknn.zip')
p3alpha_recommender.load_model(folder_path=folder_path,file_name='best_rp3alpha.zip')
p3alpha_stacked_recommender.load_model(folder_path=folder_path,file_name='best_stacked_rp3alpha.zip')
ials_recommender.load_model(folder_path=folder_path,file_name='best_ials.zip')
ials_stacked_recommender.load_model(folder_path=folder_path,file_name='best_ials_stacked.zip')
nmf_recommender.load_model(folder_path=folder_path,file_name='best_NMF.zip')

SLIMElasticNetRecommender: Loading model from file 'models/train/SLIMElasticNetRecommender.zip'
SLIMElasticNetRecommender: Loading complete
PureSVDItemRecommender: Loading model from file 'models/train/best_pure_svd.zip'
PureSVDItemRecommender: Loading complete
ScaledPureSVDRecommender: Loading model from file 'models/train/best_scaled_svd.zip'
ScaledPureSVDRecommender: Loading complete
RP3betaRecommender: Loading model from file 'models/train/best_rp3beta.zip'
RP3betaRecommender: Loading complete
RP3betaRecommender: Loading model from file 'models/train/best_stacked_rp3beta.zip'
RP3betaRecommender: Loading complete
ItemKNNCFRecommender: Loading model from file 'models/train/best_itemknn.zip'
ItemKNNCFRecommender: Loading complete
P3alphaRecommender: Loading model from file 'models/train/best_rp3alpha.zip'
P3alphaRecommender: Loading complete
P3alphaRecommender: Loading model from file 'models/train/best_stacked_rp3alpha.zip'
P3alphaRecommender: Loading complete
IALSRecommender: Loadin

In [7]:
recommenders_list={}

recommenders_list['slim']=slim_recommender
recommenders_list['pure_svd']=pure_svd_recommender
recommenders_list['scaled_svd']=scaled_svd_recommender
recommenders_list['rp3beta']=rp3beta_recommender
recommenders_list['rp3beta_stacked']=rp3beta_stacked_recommender
recommenders_list['multVAE']=multVAE_recommender

recommenders_list['itemKNN']=itemKNN_recommender
recommenders_list['p3alpha']=p3alpha_recommender
recommenders_list['p3alpha_stacked']=p3alpha_stacked_recommender
recommenders_list['ials']=ials_recommender
recommenders_list['ials_stacked']=ials_stacked_recommender
recommenders_list['nmf']=nmf_recommender




recommenders=[
recommenders_list['slim'],
recommenders_list['rp3beta'],
recommenders_list['rp3beta_stacked'],
recommenders_list['itemKNN'],
recommenders_list['p3alpha'],
recommenders_list['p3alpha_stacked'],
recommenders_list['pure_svd'],
recommenders_list['scaled_svd'],
recommenders_list['multVAE'],
recommenders_list['ials'],
recommenders_list['ials_stacked'],
recommenders_list['nmf']
]

In [8]:
from src.Recommenders.Hybrids.LinearWeightedRecommender import LinearWeightedRecommender
def objective(trial):
    weights = [
        trial.suggest_float('slim', 1e-2, 1, log=True),

        trial.suggest_float('rp3beta', 1e-3, 1, log=True),
        trial.suggest_float('rp3beta_stacked', 1e-3, 1, log=True),

        trial.suggest_float('itemKNN', 1e-3,1, log=True),
        trial.suggest_float('p3alpha', 1e-3, 1, log=True),
        trial.suggest_float('p3alpha_stacked', 1e-3, 1, log=True),


        trial.suggest_float('pure_svd', 1e-4, 0.2, log=True),
        trial.suggest_float('scaled_svd', 1e-4, 0.2, log=True),
        trial.suggest_float('multVAE', 1e-4, 0.2, log=True),
        trial.suggest_float('ials', 1e-4, 0.2, log=True),
        trial.suggest_float('ials_stacked', 1e-4, 0.2, log=True),

        trial.suggest_float('nmf', 1e-5, 0.2, log=True)


    ]

    evaluator=EvaluatorHoldout(urm_test, cutoff_list=[10])
    linear_combinator = LinearWeightedRecommender(urm_train, recommenders, weights)
    result_df, _ = evaluator.evaluateRecommender(linear_combinator)
    return result_df.loc[10, "MAP"]

In [11]:

import optuna

sampler = optuna.samplers.TPESampler(multivariate=True, group=True)
study=optuna.load_study(
    study_name='hyperparameters_hybrid_linear',
    storage=f'sqlite:///{database_path}'
)
study.sampler=optuna.samplers.CmaEsSampler(seed=42)


C:\Users\jihad\anaconda3\envs\RecSysFrameworkk\lib\site-packages\optuna\_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
C:\Users\jihad\anaconda3\envs\RecSysFrameworkk\lib\site-packages\optuna\_experimental.py:31: ExperimentalWarning: Argument ``group`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [9]:
!pip install cmaes


^C


In [12]:
study.optimize(objective, n_trials=100, n_jobs=4)

EvaluatorHoldout: Ignoring 138 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 138 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 138 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 138 ( 0.4%) Users that have less than 1 test interactions


C:\Users\jihad\PycharmProjects\RecSys2024\Recommenders\Neural\MultVAE_PyTorch_Recommender.py:188: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\SparseCsrTensorImpl.cpp:56.)
  user_batch_tensor = torch.sparse_csr_tensor(user_batch_tensor.indptr,


EvaluatorHoldout: Processed 8000 (22.5%) in 5.06 min. Users per second: 26
EvaluatorHoldout: Processed 8000 (22.5%) in 5.20 min. Users per second: 26
EvaluatorHoldout: Processed 9000 (25.3%) in 5.23 min. Users per second: 29
EvaluatorHoldout: Processed 8000 (22.5%) in 5.24 min. Users per second: 25
EvaluatorHoldout: Processed 20000 (56.2%) in 10.44 min. Users per second: 32
EvaluatorHoldout: Processed 20000 (56.2%) in 10.45 min. Users per second: 32
EvaluatorHoldout: Processed 20000 (56.2%) in 10.49 min. Users per second: 32
EvaluatorHoldout: Processed 23000 (64.6%) in 10.50 min. Users per second: 36
EvaluatorHoldout: Processed 35598 (100.0%) in 15.28 min. Users per second: 39


[I 2025-01-05 08:28:08,420] Trial 5 finished with value: 0.061627792313994754 and parameters: {'slim': 0.09143464945564576, 'rp3beta': 0.08111315273277504, 'rp3beta_stacked': 0.015439812274796762, 'itemKNN': 0.03671011021573329, 'p3alpha': 0.0128099123042032, 'p3alpha_stacked': 0.014033205606352505, 'pure_svd': 0.003452748394424517, 'scaled_svd': 0.002822275068994042, 'multVAE': 0.003672253648565988, 'ials': 0.00986628290603013, 'ials_stacked': 0.004635553911556021, 'nmf': 0.00040707791252191214}. Best is trial 80 with value: 0.06317071161479368.


EvaluatorHoldout: Ignoring 138 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 30000 (84.3%) in 15.46 min. Users per second: 32
EvaluatorHoldout: Processed 31000 (87.1%) in 15.85 min. Users per second: 33
EvaluatorHoldout: Processed 31000 (87.1%) in 15.95 min. Users per second: 32
EvaluatorHoldout: Processed 35598 (100.0%) in 17.71 min. Users per second: 34


...

EvaluatorHoldout: Processed 35598 (100.0%) in 15.21 min. Users per second: 39


[I 2025-01-05 15:02:09,435] Trial 209 finished with value: 0.06275543725926966 and parameters: {'slim': 0.41970145977493495, 'rp3beta': 0.11929687947179193, 'rp3beta_stacked': 0.0014665746643465502, 'itemKNN': 0.009612671621393643, 'p3alpha': 0.039518374263313436, 'p3alpha_stacked': 0.1374621903714903, 'pure_svd': 0.003979568471620705, 'scaled_svd': 0.007003743106421297, 'multVAE': 0.008510387267343815, 'ials': 0.0031887989241610004, 'ials_stacked': 0.005125905461666044, 'nmf': 0.0011797159263009422}. Best is trial 136 with value: 0.0632289557019858.


EvaluatorHoldout: Processed 35598 (100.0%) in 15.06 min. Users per second: 39


[I 2025-01-05 15:02:55,176] Trial 210 finished with value: 0.06312531379950695 and parameters: {'slim': 0.9468869508873056, 'rp3beta': 0.04421373156199873, 'rp3beta_stacked': 0.0012646900254694624, 'itemKNN': 0.00835592462419667, 'p3alpha': 0.055036862604690304, 'p3alpha_stacked': 0.1594037580133421, 'pure_svd': 0.003671955225641817, 'scaled_svd': 0.011455575532642398, 'multVAE': 0.018377118559344433, 'ials': 0.000978617022774413, 'ials_stacked': 0.003244818809943069, 'nmf': 0.0004144007322468194}. Best is trial 136 with value: 0.0632289557019858.


KeyboardInterrupt: 

In [ ]:
study.optimize(objective, n_trials=100, n_jobs=4)

EvaluatorHoldout: Ignoring 138 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 138 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 138 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 138 ( 0.4%) Users that have less than 1 test interactions
